In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
pip install numpy opencv-python scikit-image pillow

In [ ]:
import cv2
import numpy as np
import os
from skimage.util import random_noise

In [ ]:
def apply_fast_fading(image):
    noise_img = random_noise(image, mode='s&p', amount=0.1)
    noise_img = np.array(255 * noise_img, dtype='uint8')
    faded_img = cv2.GaussianBlur(noise_img, (5, 5), 0)
    return faded_img

In [ ]:
def create_disk_kernel(radius):
    size = 2 * radius + 1
    kernel = np.zeros((size, size), dtype=np.float32)
    center = radius
    for y in range(size):
        for x in range(size):
            if np.sqrt((x - center) ** 2 + (y - center) ** 2) <= radius:
                kernel[y, x] = 1
    return kernel / np.sum(kernel)

In [ ]:
def apply_blur(image, level=5):
    if level < 0 or level > 10:
        raise ValueError("Level should be between 0 and 10")

    kernel = create_disk_kernel(level)
    blurred = cv2.filter2D(image, -1, kernel)
    return blurred

In [ ]:
def apply_jpeg_compression(image, quality=50):
    temp_filename = 'img_aux.jpg'
    output_filename = 'rest.png'

    encode_param = [int(cv2.IMWRITE_JPEG_QUALITY), quality]
    cv2.imwrite(temp_filename, image, encode_param)

    result = cv2.imread(temp_filename)

    cv2.imwrite(output_filename, result)

    os.remove(temp_filename)
    os.remove(output_filename)

    return result

In [ ]:
def apply_white_gaussian_noise(image, level=0.1):
    if level < 0 or level > 0.5:
        raise ValueError("Level should be between 0 and 0.5")

    row, col, ch = image.shape
    mean = 0
    sigma = np.sqrt(level)
    gauss = np.random.normal(mean, sigma, (row, col, ch))
    noisy = image + gauss * 255  # Scale noise to the image's range
    noisy = np.clip(noisy, 0, 255).astype(np.uint8)
    return noisy

In [ ]:
def load_and_split_image(image_path):
    image = cv2.imread(image_path)
    h, w, _ = image.shape
    left_image = image[:, :w//2]
    right_image = image[:, w//2:]
    return left_image, right_image

In [ ]:
def apply_distortions_and_save_images():
    source_dir = '/content/drive/MyDrive/3DMX22/Images-J2K Noise/Sources'
    output_base_dir = '/content/drive/MyDrive/3DMX22/Images-Distorted'

    distortions = {
        'fast_fading': apply_fast_fading,
        'gaussian_blur': apply_gaussian_blur,
        'jpeg': apply_jpeg_compression,
        'white_gaussian_noise': apply_white_gaussian_noise,
        'disk_blur': apply_blur
    }

    for distortion_name in distortions:
        os.makedirs(os.path.join(output_base_dir, distortion_name), exist_ok=True)

    for i in range(1, 21):
        image_path = os.path.join(source_dir, f'{i:02d}.bmp')
        left_image, right_image = load_and_split_image(image_path)

        for distortion_name, distortion_func in distortions.items():
            for level in range(1, 6):  # Levels 1 to 5
                if distortion_name == 'gaussian_blur':
                    left_distorted = distortion_func(left_image, sigma=level*0.5)
                    right_distorted = distortion_func(right_image, sigma=level*0.5)
                elif distortion_name == 'jpeg':
                    left_distorted = distortion_func(left_image, quality=100 - level*10)
                    right_distorted = distortion_func(right_image, quality=100 - level*10)
                elif distortion_name == 'white_gaussian_noise':
                    left_distorted = distortion_func(left_image, level=level*0.1)
                    right_distorted = distortion_func(right_image, level=level*0.1)
                elif distortion_name == 'disk_blur':
                    left_distorted = distortion_func(left_image, level=level)
                    right_distorted = distortion_func(right_image, level=level)
                else:
                    left_distorted = distortion_func(left_image)
                    right_distorted = distortion_func(right_image)

                # Combine left and right distorted images horizontally
                combined_distorted = np.hstack((left_distorted, right_distorted))

                # Save the distorted image
                output_path = os.path.join(output_base_dir, distortion_name, f'{i:02d}_{level:02d}.bmp')
                cv2.imwrite(output_path, combined_distorted)

In [ ]:
apply_distortions_and_save_images()

In [ ]:
pip install opencv-python-headless pandas scikit-image

In [ ]:
import os
import cv2
import pandas as pd
from skimage.metrics import structural_similarity as ssim

In [ ]:
import os
import cv2
import pandas as pd
from skimage.metrics import structural_similarity as ssim

def load_image_pairs(image_path):
    """Load and split image pairs into left and right views."""
    image = cv2.imread(image_path)
    height, width = image.shape[:2]
    left = image[:, :width//2]
    right = image[:, width//2:]
    return left, right

def compute_ssim(image_left, image_right, reference_left, reference_right):
    # Define a small win_size for small images
    win_size = 5  # Choose a suitable odd number

    # Compute SSIM for left and right views with specified win_size
    ssim_left, _ = ssim(image_left, reference_left, win_size=win_size, full=True, channel_axis=-1)
    ssim_right, _ = ssim(image_right, reference_right, win_size=win_size, full=True, channel_axis=-1)

    # Average SSIM score
    score = (ssim_left + ssim_right) / 2.0
    return score


def process_images(reference_path, distortions_paths):
    """Compute SSIM scores for all distorted images relative to reference images."""
    results = []

    # Load reference images
    reference_images = sorted([f for f in os.listdir(reference_path) if f.endswith('.bmp')])

    for ref_image_name in reference_images:
        ref_image_path = os.path.join(reference_path, ref_image_name)
        ref_left, ref_right = load_image_pairs(ref_image_path)

        for distortion_type, distortion_path in distortions_paths.items():
            distortion_images = sorted([f for f in os.listdir(distortion_path) if f.endswith('.bmp')])

            for dist_image_name in distortion_images:
                if dist_image_name.startswith(ref_image_name.split('.')[0]):
                    dist_image_path = os.path.join(distortion_path, dist_image_name)
                    dist_left, dist_right = load_image_pairs(dist_image_path)
                    score = compute_ssim(dist_left, dist_right, ref_left, ref_right)

                    image_name = os.path.splitext(dist_image_name)[0]  # Remove extension
                    results.append([distortion_type, image_name, score])

    return results

# Define paths
reference_path = '/content/drive/MyDrive/3DMX22/Images-J2K Noise/Sources/'
distortions_paths = {
    'disk_blur': '/content/drive/MyDrive/3DMX22/Images-Distorted/disk_blur/',
    'fast_fading': '/content/drive/MyDrive/3DMX22/Images-Distorted/fast_fading/',
    'gaussian_blur': '/content/drive/MyDrive/3DMX22/Images-Distorted/gaussian_blur/',
    'jpeg': '/content/drive/MyDrive/3DMX22/Images-Distorted/jpeg/',
    'jpeg2k': '/content/drive/MyDrive/3DMX22/Images-Distorted/jpeg2k/',
    'white_gaussian_noise': '/content/drive/MyDrive/3DMX22/Images-Distorted/white_gaussian_noise/'
}

# Process and save results
results = process_images(reference_path, distortions_paths)
df = pd.DataFrame(results, columns=['distortion type', 'image name', 'ssim score'])
df.to_csv('/content/ssim_scores.csv', index=False)
